In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
import os
import cv2
import time
import glob
import math
import re
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline

import imutils as imutils
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # machine learning
from tqdm import tqdm # make your loops show a smart progress meter 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sn
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
from keras.models import model_from_json

In [ ]:
DATADIR = "/content/gdrive/My Drive/thesis/"

In [ ]:
# load json and create model
json_file = open("/content/gdrive/My Drive/thesis/Model_5/cnn_3_1_results/model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/gdrive/My Drive/thesis/Model_5/cnn_3_1_results/model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
loaded_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Loaded model from disk


rest of pipeline

In [ ]:
DATADIR = "/content/gdrive/My Drive/thesis/"

getting image data from the directory

In [ ]:
# to sort the files by their frame number
def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    l.sort(key=alphanum_key)

In [ ]:
# returns filenames from the folder path
def get_all_camera_files(img_dir):
  data_path = os.path.join(img_dir,'*g')
  files = glob.glob(data_path)
  sort_nicely(files)

  return files

img_dir = DATADIR + 'paper_results/'
filenames = get_all_camera_files(img_dir)

In [ ]:
len(filenames)

55

YOLO for bounding boxes

In [ ]:
coco_list = [0]#, 2, 5, 7] # person, car, bus, truck

In [ ]:
# filepath for the dependent files; this will change based on the original file path
yolomodel = {"config_path": "/content/gdrive/My Drive/thesis/YOLO/yolov3.cfg",
              "model_weights_path": "/content/gdrive/My Drive/thesis/YOLO/yolov3.weights",
              "coco_names": "/content/gdrive/My Drive/thesis/YOLO/coco.names",
              "confidence_threshold": 0.7,
              "threshold":0.3
             }

net = cv2.dnn.readNetFromDarknet(yolomodel["config_path"], yolomodel["model_weights_path"])
labels = open(yolomodel["coco_names"]).read().strip().split("\n")

In [ ]:
len(net.getLayerNames())

254

In [ ]:
np.random.seed(12345)
layer_names = net.getLayerNames()
layer_names = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

In [ ]:
# returns the bounding box details along with the bounding box displayed on the frame
def object_detection(frame):
    global coco_list
    (H, W) = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections_layer = net.forward(layer_names)   # detect objects using object detection model
    
    boxes, confidences, classIDs = [], [], []
    for out in detections_layer:
        for detection in out:
            scores = detection[5:]
            classID = np.argmax(scores)
            if classID not in coco_list:
                continue
            confidence = scores[classID]
            
            if confidence > yolomodel['confidence_threshold']:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, yolomodel["confidence_threshold"], yolomodel["threshold"])
    
    nms_boxes = []
    nms_confidences = []
    nms_classIDs = []
    
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
                        
            # clr = [int(c) for c in bbox_colors[classIDs[i]]]
            # cv2.rectangle(frame, (x, y), (x+w, y+h), clr, 2)
            nms_boxes.append([x, y, w, h])
            nms_confidences.append(confidences[i])
            nms_classIDs.append(classIDs[i])
            
    return nms_boxes, nms_confidences, nms_classIDs

In [ ]:
def plot_boxes(frame, detections):
    for i in range(len(detections)):
      bbox = detections[i]
      [x, y, w, h] = bbox
      clr=(255,255,255)
      cv2.rectangle(frame, (x, y), (x+w, y+h), clr, 3)
    return frame

In [ ]:
def enlarge_bounding_boxes(orig_bboxes):
    modf_bboxes = []
    for bbox in orig_bboxes:
        [x1, y1, w1, h1] = bbox
        [x2, y2, w2, h2] = [x1-w1, y1, 3*w1, h1+h1//4]
        if x2<0: x2=0
        if y2<0: y2=0
        
        modf_bboxes.append([x2, y2, w2, h2])
    return modf_bboxes

In [ ]:
def plot_detections(frame, detections, predictions):
    for i in range(len(detections)):
      bbox = detections[i]
      pred = predictions[i]
      [x, y, w, h] = bbox
      if pred==0: clr=(0,255,0)
      else: clr=(0,255,255)

      overlay = frame.copy()
      cv2.rectangle(overlay, (x, y), (x+w, y+h), clr, -1)
      alpha = 0.5
      cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    return frame

for a scenario

In [ ]:
start = time.time()
writer = None
(W,H) = (None, None)

IMG_SIZE = 100
fps = 10
out_dir = DATADIR+'scenario/esc_det_output.avi'
count = 0

for filename in filenames:
  frame = cv2.imread(filename)
  if W is None or H is None: (H, W) = frame.shape[:2]

  detections, _, _ = object_detection(frame)
  enl_bboxes = enlarge_bounding_boxes(detections)

  x_frame = []
  predictions = []

  for i in range(len(enl_bboxes)):
    bbox = enl_bboxes[i]
    [x, y, w, h] = bbox
    extracted_frame = frame[y:y+h,x:x+w,:]
    img_arr = cv2.resize(extracted_frame, (IMG_SIZE, IMG_SIZE))
    x_frame.append(img_arr/255.0)

  if x_frame!=[]:
    X = np.asarray(x_frame)
    predictions = np.rint(loaded_model.predict(X)) #calculate confidence here

  frame = plot_detections(frame, detections, predictions)

  if writer is None:
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(out_dir, fourcc, fps, (W,H), True)
  writer.write(frame)
  print("Frame: ", count+1)
  count = count+1

writer.release()

end = time.time()
print("Total time taken: ", end - start, "seconds")

for individual unique frames

In [ ]:
IMG_SIZE = 100
out_dir = DATADIR+'paper_results/output/'
count = 0

for filename in filenames:
  frame = cv2.imread(filename)
  detections, _, _ = object_detection(frame)
  enl_bboxes = enlarge_bounding_boxes(detections)

  x_frame = []
  predictions = []

  for i in range(len(enl_bboxes)):
    bbox = enl_bboxes[i]
    [x, y, w, h] = bbox
    extracted_frame = frame[y:y+h,x:x+w,:]
    img_arr = cv2.resize(extracted_frame, (IMG_SIZE, IMG_SIZE))
    x_frame.append(img_arr/255.0)

  if x_frame!=[]:
    X = np.asarray(x_frame)
    predictions = np.rint(loaded_model.predict(X)) #calculate confidence here

  frame = plot_detections(frame, detections, predictions)

  # save the frame
  # im = Image.fromarray(frame)  
  if not os.path.exists(out_dir):
    os.makedirs(out_dir)
  fnm = out_dir + re.split('/',filename)[-1]
  # im.save(fnm)
  cv2.imwrite(fnm, frame)
  print("Frame: ", count+1)
  count = count+1

Frame:  1
Frame:  2
Frame:  3
Frame:  4
Frame:  5
Frame:  6
Frame:  7
Frame:  8
Frame:  9
Frame:  10
Frame:  11
Frame:  12
Frame:  13
Frame:  14
Frame:  15
Frame:  16
Frame:  17
Frame:  18
Frame:  19
Frame:  20
Frame:  21
Frame:  22
Frame:  23
Frame:  24
Frame:  25
Frame:  26
Frame:  27
Frame:  28
Frame:  29
Frame:  30
Frame:  31
Frame:  32
Frame:  33
Frame:  34
Frame:  35
Frame:  36
Frame:  37
Frame:  38
Frame:  39
Frame:  40
Frame:  41
Frame:  42
Frame:  43
Frame:  44
Frame:  45
Frame:  46
Frame:  47
Frame:  48
Frame:  49
Frame:  50
Frame:  51
Frame:  52
Frame:  53
Frame:  54
Frame:  55


In [ ]:
# filename
re.split('/',filename)[-1]

'frame_34_1601148902071558987.jpg'